In [1]:
import os
from groq import Groq
import re
from typing import List, Dict, Any
import yaml
from tqdm import tqdm
import json
from openai import OpenAI

In [ ]:
API_KEY = ""
DJANGO_PROJECT_PATH = "/Users/ryanmarr/Documents/saleor"
MODEL_NAME = "gpt-4o"
MAX_TOKENS = 2048
TEMPERATURE = 0.1
TOP_P = 1
STREAM = False

In [3]:
# Groq client setup
client = OpenAI(api_key=API_KEY)

In [4]:
def find_django_files(directory: str) -> List[str]:
    """Perform DFS to find all Python files in Django project"""
    django_files = []
    priority_files = []

    for root, dirs, files in os.walk(directory):
        # Skip common directories that don't contain Django code
        dirs[:] = [d for d in dirs if not d.startswith('.') and d not in ['node_modules', '__pycache__', 'venv', 'env', '.git']]
        
        for file in files:
            if file == 'views.py':
                priority_files.append(os.path.join(root, file))
            elif file.endswith('.py'):
                file_path = os.path.join(root, file)
                django_files.append(file_path)

    return priority_files + django_files

In [5]:
# def extract_rest_apis_from_file_with_openai(file_path: str) -> List[Dict[str, Any]]:
#     """Use OpenAI to analyze file content and find REST APIs"""
#     try:
#         with open(file_path, 'r', encoding='utf-8') as f:
#             content = f.read()  
        
#         # Skip files that don't contain common Django/API keywords
#         if not any(keyword in content.lower() for keyword in ['api', 'view', 'rest', 'http', 'request', 'response', 'serializer']):
#             return []

#         prompt = f"""
#         Analyze this Python file and identify all Django API functions or classes. Only return the functions that are API endpoints. Only return entire implemenation of the function including function signature and content. 
        
#         File: {file_path}
        
#         Look for:
#         1. Functions that handle HTTP methods (GET, POST, PUT, DELETE, PATCH)
#         2. Functions that process requests and return responses
        
#         File content:
#         {content}  
        
#         IMPORTANT: Return ONLY valid JSON with this exact structure:
#         {{
#             "apis": [
#                 {{
#                     "name": "function_or_class_name",
#                     "http_method": "GET|POST|PUT|DELETE|PATCH|UNKNOWN",
#                     "description": "Brief description of what this API does",
#                     "content_django": actual function and entire implementation funtion include function signature and content,
#                     "content_dafny": Based on the django function, create a Dafny function specification with preconditions and postconditions assume db schema exists as a dafny type
#                 }}
#             ]
#         }}
        
#         If no APIs are found, return: {{"apis": []}}
#         Do not include any text before or after the JSON.
#         """
        
#         try:
#             completion = client.chat.completions.create(
#                 model=MODEL_NAME,
#                 messages=[
#                     {
#                         "role": "user",
#                         "content": prompt
#                     }
#                 ],
#                 temperature=TEMPERATURE,
#                 max_tokens=MAX_TOKENS,
#                 top_p=TOP_P,
#                 stream=STREAM
#             )
            
#             response = completion.choices[0].message.content.strip()
            
#             # Improved JSON extraction
#             try:
#                 # First, try to parse the entire response as JSON
#                 result = json.loads(response)
#                 if 'apis' in result:
#                     for api in result.get('apis', []):
#                         api['file_path'] = file_path
#                     return result.get('apis', [])
#             except json.JSONDecodeError:
#                 # If that fails, try to find JSON within the response
#                 # Look for content between the first { and last }
#                 start = response.find('{')
#                 end = response.rfind('}')
                
#                 if start != -1 and end != -1 and end > start:
#                     json_str = response[start:end + 1]
#                     try:
#                         result = json.loads(json_str)
#                         if 'apis' in result:
#                             for api in result.get('apis', []):
#                                 api['file_path'] = file_path
#                             return result.get('apis', [])
#                     except json.JSONDecodeError:
#                         print(f"Failed to parse extracted JSON from response for {file_path}")
#                         print(f"Extracted JSON string: {json_str}")
#                         return []
#                 else:
#                     print(f"No JSON structure found in response for {file_path}")
#                     print(f"Response: {response}")
#                     return []
                
#         except Exception as e:
#             print(f"Error calling OpenAI API for {file_path}: {e}")
#             return []
            
#     except Exception as e:
#         print(f"Error reading file {file_path}: {e}")
#         return []

In [6]:
def extract_rest_apis_from_file_with_openai(file_path: str) -> List[Dict[str, Any]]:
    """Use OpenAI to analyze file content and find REST APIs"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()  
        
        # Skip files that don't contain common Django/API keywords
        if not any(keyword in content.lower() for keyword in ['api', 'view', 'rest', 'http', 'request', 'response', 'serializer']):
            return []

        prompt = f"""
        Analyze this Python file and identify all Django REST API functions or classes. Only return the functions that are REST API endpoints. Only return entire implemenation of the function including function signature and content. 
        
        File: {file_path}
        
        Look for:
        1. Functions that handle HTTP methods (GET, POST, PUT, DELETE, PATCH)
        2. Functions that process requests and return responses
        3. Any other REST API endpoints
        
        File content:
        {content}  # First 4000 chars for analysis
        
        IMPORTANT: Return ONLY valid JSON with this exact structure:
        {{
            "apis": [
                {{
                    "name": "function_or_class_name",
                    "http_method": "GET|POST|PUT|DELETE|PATCH|UNKNOWN",
                    "description": "Brief description of what this API does",
                    "content_django": actual function and entire implementation funtion include function signature and content,
                    "content_dafny": Based on the django function, create a Dafny function specification with preconditions and postconditions assume db schema exists as a dafny type
                }}
            ]
        }}
        
        If no REST APIs are found, return: {{"apis": []}}
        Do not include any text before or after the JSON.
        """
        
        try:
            completion = client.chat.completions.create(
                model=MODEL_NAME,
                messages=[
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                temperature=TEMPERATURE,
                max_tokens=MAX_TOKENS,
                top_p=TOP_P,
                stream=STREAM
            )
            
            response = completion.choices[0].message.content.strip()
            
            # Improved JSON extraction
            try:
                # First, try to parse the entire response as JSON
                result = json.loads(response)
                if 'apis' in result:
                    for api in result.get('apis', []):
                        api['file_path'] = file_path
                    return result.get('apis', [])
            except json.JSONDecodeError:
                # If that fails, try to find JSON within the response
                # Look for content between the first { and last }
                start = response.find('{')
                end = response.rfind('}')
                
                if start != -1 and end != -1 and end > start:
                    json_str = response[start:end + 1]
                    try:
                        result = json.loads(json_str)
                        if 'apis' in result:
                            for api in result.get('apis', []):
                                api['file_path'] = file_path
                            return result.get('apis', [])
                    except json.JSONDecodeError:
                        print(f"Failed to parse extracted JSON from response for {file_path}")
                        print(f"Extracted JSON string: {json_str}")
                        return []
                else:
                    print(f"No JSON structure found in response for {file_path}")
                    print(f"Response: {response}")
                    return []
                
        except Exception as e:
            print(f"Error calling OpenAI API for {file_path}: {e}")
            return []
            
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return []

In [7]:
"""Analyze Django project and generate Dafny specifications"""

# Find all Python files
django_files = find_django_files(DJANGO_PROJECT_PATH)
print(f"Found {len(django_files)} Python files")

# Extract REST APIs using OpenAI
all_apis = []
for file_path in tqdm(django_files, desc="Analyzing files"):
    apis = extract_rest_apis_from_file_with_openai(file_path)
    print(f'apis: {apis}')
    all_apis.extend(apis)

Found 4045 Python files


Analyzing files:   0%|          | 1/4045 [00:30<33:56:38, 30.22s/it]

apis: [{'name': 'handle_thumbnail', 'http_method': 'UNKNOWN', 'description': 'Create and return thumbnail for given instance in provided size and format.', 'content_django': 'def handle_thumbnail(request, instance_id: str, size: str, format: str | None = None):\n    """Create and return thumbnail for given instance in provided size and format.\n\n    If the provided size is not in the available resolution list, the thumbnail with\n    the closest available size is created and returned, if it does not exist.\n    """\n    # try to find corresponding instance based on given instance_id\n    try:\n        object_type, pk = from_global_id_or_error(instance_id, raise_error=True)\n    except GraphQLError:\n        return HttpResponseNotFound("Cannot found instance with the given id.")\n\n    if object_type not in TYPE_TO_MODEL_DATA_MAPPING.keys():\n        return HttpResponseNotFound("Invalid instance type.")\n\n    # check formats\n    format = format.lower() if format else None\n    if obj

Analyzing files:   0%|          | 2/4045 [00:33<15:51:00, 14.11s/it]

apis: [{'name': 'jwks', 'http_method': 'GET', 'description': 'Returns JSON Web Key Set (JWKS) for JWT validation.', 'content_django': 'def jwks(request):\n    return JsonResponse(get_jwt_manager().get_jwks())', 'content_dafny': 'method jwks(request: Request) returns (response: JsonResponse)\n  requires request != null\n  ensures response != null\n  ensures response.content == get_jwt_manager().get_jwks()', 'file_path': '/Users/ryanmarr/Documents/saleor/saleor/core/views.py'}]


...

apis: []


Analyzing files:   2%|▏         | 97/4045 [04:56<5:21:27,  4.89s/it]

apis: []


Analyzing files:   2%|▏         | 97/4045 [04:57<3:21:50,  3.07s/it]


KeyboardInterrupt: 

In [8]:
def display_full_api_info(apis: List[Dict[str, Any]]):
    """Display complete API information with full Django code and Dafny specs"""
    
    print("=" * 100)
    print("COMPLETE DJANGO REST API ANALYSIS")
    print("=" * 100)
    print(f"Total APIs found: {len(apis)}")
    print()
    
    # Group by file path
    files = {}
    for api in apis:
        file_path = api.get('file_path', 'Unknown')
        if file_path not in files:
            files[file_path] = []
        files[file_path].append(api)
    
    # Display by file
    for file_path, file_apis in files.items():
        print(f"📁 FILE: {file_path}")
        print("=" * 100)
        print(f"APIs found: {len(file_apis)}")
        print()
        
        for i, api in enumerate(file_apis, 1):
            print(f"🔗 API #{i}: {api.get('name', 'Unknown')}")
            print(f"   Method: {api.get('http_method', 'UNKNOWN')}")
            print(f"   Description: {api.get('description', 'No description')}")
            print()
            
            print("📝 DJANGO CODE:")
            print("-" * 50)
            django_code = api.get('content_django', 'No Django code available')
            print(django_code)
            print()
            
            print("🔬 DAFNY SPECIFICATION:")
            print("-" * 50)
            dafny_spec = api.get('content_dafny', 'No Dafny specification available')
            print(dafny_spec)
            print()
            
            print("─" * 100)
            print()
    
    # Summary statistics
    print("=" * 100)
    print("SUMMARY STATISTICS")
    print("=" * 100)
    
    # HTTP method distribution
    methods = {}
    for api in apis:
        method = api.get('http_method', 'UNKNOWN')
        methods[method] = methods.get(method, 0) + 1
    
    print("HTTP Methods:")
    for method, count in sorted(methods.items()):
        print(f"  {method}: {count}")
    
    print()
    print(f"Files analyzed: {len(files)}")
    print(f"Total APIs: {len(apis)}")

In [9]:
display_full_api_info(all_apis)

COMPLETE DJANGO REST API ANALYSIS
Total APIs found: 8

📁 FILE: /Users/ryanmarr/Documents/saleor/saleor/thumbnail/views.py
APIs found: 1

🔗 API #1: handle_thumbnail
   Method: UNKNOWN
   Description: Create and return thumbnail for given instance in provided size and format.

📝 DJANGO CODE:
--------------------------------------------------
def handle_thumbnail(request, instance_id: str, size: str, format: str | None = None):
    """Create and return thumbnail for given instance in provided size and format.

    If the provided size is not in the available resolution list, the thumbnail with
    the closest available size is created and returned, if it does not exist.
    """
    # try to find corresponding instance based on given instance_id
    try:
        object_type, pk = from_global_id_or_error(instance_id, raise_error=True)
    except GraphQLError:
        return HttpResponseNotFound("Cannot found instance with the given id.")

    if object_type not in TYPE_TO_MODEL_DATA_MAPPING